# Pick and Place

This tutorial shows the use of Pick & Place tasks.

Initialize the robot and environment.

In [1]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.pose import Pose
from pycram.bullet_world import BulletWorld, Object
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

world = BulletWorld()
broadcaster = TFBroadcaster()
v = VizMarkerPublisher()
robot = Object("pr2", "robot", "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", "environment", "apartment.urdf")

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown tag "material" in /robot[@name='plane']/link[@name='planeLink']/collision[1]
Unknown tag "contact" in /robot[@name='plane']/link[@name='planeLink']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "t

Spawn objects, attach spoon to the drawer, and set local variables to the robot, apartment and standard pose.

In [2]:
milk = Object("milk", "milk", "milk.stl", pose=Pose([2.5, 2, 1.02]), color=[1, 0, 0, 1])
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", pose=Pose([2.5, 2.3, 1.05]), color=[0, 1, 0, 1])
spoon = Object("spoon", "spoon", "spoon.stl", pose=Pose([2.4, 2.2, 0.85]), color=[0, 0, 1, 1])
bowl = Object("bowl", "bowl", "bowl.stl", pose=Pose([2.5, 2.2, 1.02]), color=[1, 1, 0, 1])
apartment.attach(spoon, 'cabinet10_drawer_top')

pick_pose = Pose([2.7, 2.15, 1])

robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

Define a local function to move, look and detect an object. The decorator `with_simulated_robot` indicates, that we stick to the simulation, instead of executing the tasks on the real robot.

In [3]:
@with_simulated_robot
def move_and_detect(obj_type):
    NavigateAction(target_locations=[Pose([1.7, 2, 0])]).resolve().perform()

    LookAtAction(targets=[pick_pose]).resolve().perform()

    object_desig = DetectAction(BelieveObject(types=[obj_type])).resolve().perform()

    return object_desig

Without the decorator we can specify the simulated robot as a block.

In [4]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.25]).resolve().perform()

First, transport the milk, using the `move_and_detect` function from above.

In [5]:
with simulated_robot:
    milk_desig = move_and_detect("milk")

    TransportAction(milk_desig, ["left"], [Pose([4.8, 3.55, 0.8])]).resolve().perform()

Now transport the cereal box.

In [ ]:
with simulated_robot:
    cereal_desig = move_and_detect("cereal")

    TransportAction(cereal_desig, ["right"], [Pose([5.2, 3.4, 0.8], [0, 0, 1, 1])]).resolve().perform()


Now the bowl.

In [ ]:
with simulated_robot:
    bowl_desig = move_and_detect("bowl")

    TransportAction(bowl_desig, ["left"], [Pose([5, 3.3, 0.8], [0, 0, 1, 1])]).resolve().perform()

For the final object we need to fetch it from a drawer. First we calculate a suitable pose to reach the drawer.

In [ ]:
# with simulated_robot:
#     # Finding and navigating to the drawer holding the spoon
#     handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())
#     drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(), robot_desig=robot_desig.resolve()).resolve()
#
#     NavigateAction([drawer_open_loc.pose]).resolve().perform()

In [5]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # #
 # # # # # # # # # # # # # # # # # # # # # # # # # # # #
with simulated_robot:
    handle_desig = ObjectPart(names=["handle_cab10_t"], part_of=apartment_desig.resolve())


In [6]:
with simulated_robot:
    drawer_open_loc_solutions = iter(AccessingLocation(handle_desig=handle_desig.resolve(), robot_desig=robot_desig.resolve()))

In [10]:
with simulated_robot:
    drawer_open_loc = next(drawer_open_loc_solutions)

In [11]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

In [12]:
with simulated_robot:
    NavigateAction([drawer_open_loc.pose]).resolve().perform()

Open the drawer. The spoon moves with the drawer, because it was attached to it. To move the spoon, we need to detach it.

In [13]:
with simulated_robot:
    OpenAction(object_designator_description=handle_desig, arms=[drawer_open_loc.arms[0]]).resolve().perform()

Look at the general direction of the drawer.

In [14]:
with simulated_robot:
    # Detect and pickup the spoon
    LookAtAction([apartment.get_link_pose("handle_cab10_t")]).resolve().perform()

Now perceive it, when it is in the field of view.

In [ ]:
with simulated_robot:
    spoon_desig = DetectAction(BelieveObject(types=["spoon"])).resolve().perform()


Lets see how the spoon looks like.

In [ ]:
spoon_desig

Now that we know where exactly the spoon is, we can pick it up.

In [ ]:
with simulated_robot:
    pickup_arm = "left" if drawer_open_loc.arms[0] == "right" else "right"
    PickUpAction(spoon_desig, [pickup_arm], ["top"]).resolve().perform()

Get the right arm back up.

In [ ]:
with simulated_robot:
    ParkArmsAction([Arms.RIGHT]).resolve().perform()

Close the drawer with the left arm still holding the handle.

In [ ]:
with simulated_robot:
    CloseAction(object_designator_description=handle_desig, arms=[drawer_open_loc.arms[0]]).resolve().perform()


Release the handle and retract the gripper.

In [ ]:
with simulated_robot:
    gripper = "left"
    motion = "open"
    SetGripperAction(grippers=[gripper], motions=[motion]).resolve().perform()
    ParkArmsAction([Arms.LEFT]).resolve().perform()

Reposition the torso.

In [ ]:
with simulated_robot:
    MoveTorsoAction([0.15]).resolve().perform()

Calculate and move to the target location where to drop the spoon.

In [ ]:
with simulated_robot:
    # Find a pose to place the spoon, move and then place it
    spoon_target_pose = Pose([4.85, 3.3, 0.8], [0, 0, 1, 1])
    placing_loc = CostmapLocation(target=spoon_target_pose, reachable_for=robot_desig.resolve()).resolve()

    NavigateAction([placing_loc.pose]).resolve().perform()

Place the spoon at the target pose.

In [ ]:
with simulated_robot:
    PlaceAction(spoon_desig, [spoon_target_pose], [pickup_arm]).resolve().perform()

Open the gripper to release the spoon and retract.

In [ ]:
with simulated_robot:
    gripper = "right"
    motion = "open"
    SetGripperAction(grippers=[gripper], motions=[motion]).resolve().perform()
    ParkArmsAction([Arms.RIGHT]).resolve().perform()